In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from transformers import AutoTokenizer, pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from datasets import load_dataset

from torch.utils.data import DataLoader

In [ ]:
path = "../trainedModels"
model_name = "roberta-base"

In [ ]:
dataset = load_dataset('csv', data_files={'train': '../transformers_data/train.csv',
                                         'validation': '../transformers_data/validation.csv',
                                         'test': '../transformers_data/test.csv'})

label_mapping = {
    "Effective": 0,
    "Adequate": 1,
    "Ineffective": 2
}

def encode_labels(example):
    example['labels'] = label_mapping[example['discourse_effectiveness']]
    return example

dataset = dataset.map(encode_labels)

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example['discourse_text'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(['discourse_id', 'essay_id', 'discourse_text', 'discourse_effectiveness'])  # Remove unnecessary columns
tokenized_datasets.set_format('torch')

train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=16)
eval_dataloader = DataLoader(tokenized_datasets['validation'], batch_size=16)

from transformers import AutoModelForSequenceClassification

num_labels = 3  # Effective, Adequate, Not Effective
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=0.1,
    weight_decay=0.02,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    save_strategy='epoch',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

trainer.train()

results = trainer.evaluate(tokenized_datasets['test'])
print(results)

Map:   0%|          | 0/3677 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/246 [00:00<?, ?it/s]

  0%|          | 0/307 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

  0%|          | 0/307 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.9761713147163391, 'eval_accuracy': 0.5705738373674191, 'eval_precision': 0.325554503888182, 'eval_recall': 0.5705738373674191, 'eval_f1': 0.4145675881547516, 'eval_runtime': 33.3716, 'eval_samples_per_second': 110.184, 'eval_steps_per_second': 9.199, 'epoch': 0.1003671970624235}


  0%|          | 0/307 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.save_pretrained(f'{path}/{model_name}')

tokenizer.save_pretrained(f'{path}/{model_name}')

# Guardar el modelo entrenado
# Save the Trainer object to a file
with open(f'{path}/{model_name}/{model_name}.pkl', 'wb') as f:
    pickle.dump(trainer.state.log_history, f)

# Make predictions
predictions = trainer.predict(tokenized_datasets['test'])

# Guardar el modelo entrenado
# Save the Trainer object to a file
with open(f'{path}/{model_name}/{model_name}-predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f)